#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

import datetime


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler)


# Добавление строки в лог
def add_log(sname):
    f = open('log.csv', 'a')
    f.write(str(sname)+','+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\n')
    f.close()
    return True

server.register_function(add_log, 'add_log')


print ("Listening on port 8018...")
server.serve_forever()


Listening on port 8018...


127.0.0.1 - - [16/Sep/2022 16:06:11] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 16:06:15] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 16:06:19] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 16:06:23] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 16:06:27] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 16:06:31] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 16:06:35] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 16:06:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 16:06:43] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 16:06:47] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 16:06:51] "POST /RPC2 HTTP/1.1" 200 -
